In [73]:
import os
os.chdir(os.path.dirname(os.path.abspath("__file__")))


In [74]:
import sys
print(sys.executable)
print(sys.path)

c:\Git_Tutorials\IntentClassification\.venv\Scripts\python.exe
['C:\\Users\\bsaar\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\bsaar\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\bsaar\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\bsaar\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Git_Tutorials\\IntentClassification\\.venv', '', 'c:\\Git_Tutorials\\IntentClassification\\.venv\\Lib\\site-packages', 'c:\\Git_Tutorials\\IntentClassification\\.venv\\Lib\\site-packages\\win32', 'c:\\Git_Tutorials\\IntentClassification\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Git_Tutorials\\IntentClassification\\.venv\\Lib\\site-packages\\Pythonwin']


In [75]:
import pandas as pd
import numpy as np
import pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
from pathlib import Path
import requests
from zipfile import ZipFile


In [76]:
# code to convert dataframe records to json
def csv_to_json_converter(csv_file_path):
    try:
        local_df = pd.read_csv(csv_file_path)
        local_df.reset_index(drop=True, inplace=True)
        records = local_df.to_dict(orient='records')
        return records
    except Exception as e:
        raise e

In [77]:
#code to insert the jsons into mongodb

def insert_data_mongodb(json_data, database, collection):
    try:
        #connect to MongoDB
        load_dotenv()
        client = MongoClient(os.getenv('MONGO_DB_URL'), server_api=ServerApi('1'))
        client.admin.command('ping')
        print("Connected to MongoDB")
        db = client[database]
        colln = db[collection]
        #insert the json data
        colln.insert_many(json_data)
        return(f"Inserted {len(json_data)} records into {collection} collection in {database} database.")
    except Exception as e:
        raise e

In [78]:
def export_collection_as_csv(csv_file_path, database, collection):
    try:
        load_dotenv()
        client = MongoClient(os.getenv('MONGO_DB_URL'), server_api=ServerApi('1'))
        client.admin.command('ping')
        print("Connected to MongoDB")
        db = client[database]
        colln=db[collection]
        
        df = pd.DataFrame(list(colln.find()))
        print("export successful")
        if "_id" in df.columns:
            df.drop(columns=['_id'], inplace =True, axis=1)
        if "Unnamed: 0" in df.columns:
            df.drop(columns=['Unnamed: 0'], inplace =True, axis=1)
        print("csv_file saved succesfully")
        df.to_csv(csv_file_path, index=False, header = True)

    except Exception as e:
        print(e)
        raise e

In [79]:
from src.IntentClassification.utils.common import read_yaml
from src.IntentClassification import constants as const
import re

sentence = "perform partial redemption using service dptdrecordarredemption"

sentence = re.sub(r'\bdp\w+','web service',sentence)
print(sentence)



perform partial redemption using service web service


In [ ]:
def download_glove_embeddings(url, glove_folder, glove_zip_file):
    if not os.path.exists(glove_folder):
        os.makedirs(glove_folder, exist_ok=True)
        response = requests.get(url, stream=True)
        with open(glove_zip_file, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded glove embeddings from {url}")
        with ZipFile(glove_zip_file, 'r') as zip:
            zip.extractall(glove_folder)
        print(f"Unzipped glove embeddings to {glove_folder}")


In [81]:
url = "http://nlp.stanford.edu/data/glove.6B.zip"
glove_folder = "artifacts/glove"
glove_zip_file = "artifacts/glove/glove.6B.zip"

download_glove_embeddings(url, glove_folder, glove_zip_file)

In [82]:
def load_glove_embeddings(file_path="glove_data/glove.6B.100d.txt"):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            word = parts[0]
            vector = np.array(parts[1:], dtype=np.float32)
            embeddings[word] = vector
    print(f"Loaded {len(embeddings)} word vectors.")
    return embeddings


In [83]:
glove_path = "artifacts/glove/glove.6B.100d.txt"

glove_embeddings = load_glove_embeddings(glove_path)

Loaded 400000 word vectors.


In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
df = pd.read_csv('artifacts/data_preprocessing/preprocessed.csv')
corpus = df['preprocessed'].astype(str).tolist()

tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
idf_scores = dict(zip(tfidf_vectorizer.get_feature_names_out(),tfidf_vectorizer.idf_))


In [85]:
def sentence_to_weighted_glove(sentence, glove_embeddings, idf_scores):
    words = sentence.split()
    vector = np.zeros(100)  # Assuming 100-dimensional GloVe embeddings
    total_weight = 0
    to_be_vectors = []

    for word in words:
        print(word)
        if word in glove_embeddings and word in idf_scores:
            weight = (idf_scores[word])
            vector += glove_embeddings[word]*weight
            total_weight += weight
        else:
            to_be_vectors.append(word)
            print(len(to_be_vectors))
            print(to_be_vectors)
    return vector / total_weight if total_weight != 0 else vector


In [87]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess


In [88]:
corpus = ['create a checking_account', 'place account_freeze restriction']

In [72]:
from nltk.tokenize import word_tokenize

def tokenize_corpus(corpus):
    return [word_tokenize(text) for text in corpus]

In [91]:
word_2_vec_path = "artifacts/word2vec.model"

def save_word2vec_model(corpus, model_path):
    model = Word2Vec(sentences=tokenize_corpus(corpus), vector_size=100, window=5, min_count=1, workers=4)
    model.save(model_path)

def load_word2vec_model(model_path):
    return Word2Vec.load(model_path)

In [92]:
save_word2vec_model(corpus, word_2_vec_path)

In [93]:
def sentence_to_weighted_vectors(sentence, glove_embeddings, idf_scores, model_path):
    words = word_tokenize(sentence)
    vector = np.zeros(100)  # Assuming 100-dimensional GloVe embeddings
    total_weight = 0

    word_2_vec = load_word2vec_model(model_path)

    for word in words:
        
        if word in glove_embeddings and word in idf_scores:
            weight = (idf_scores[word])
            vector += glove_embeddings[word]*weight
            total_weight += weight
        else:
            print(word)
            vector += word_2_vec.wv[word]
    return vector / total_weight if total_weight != 0 else vector

In [98]:
vec = {}
for sentence in corpus:
    vector = sentence_to_weighted_vectors(sentence, glove_embeddings, idf_scores, word_2_vec_path)
    vec[sentence] = vector

a
checking_account
account_freeze


In [99]:
for item, value in vec.items():
    print(f"Sentence: {item}\nVector: {value}\n")

Sentence: create a checking_account
Vector: [-2.68656872e-01  2.67037610e-01 -3.51864134e-01  8.19922793e-02
  9.10801804e-01 -4.83896640e-02 -4.56233171e-01 -2.22512343e-01
  3.05263497e-01  3.32670408e-01 -4.67534921e-01 -2.08155619e-01
 -1.91024881e-01 -6.11630179e-01 -4.72010342e-02 -1.21737681e-01
  4.24713741e-01  5.06791987e-01 -3.23761464e-01  1.66680504e-01
 -1.51868938e-02 -4.85195918e-01  4.69546150e-01 -5.55205749e-01
 -2.81407719e-01 -6.52552732e-01 -1.73183683e-01 -2.77245815e-01
 -1.56585879e-01 -4.37331469e-01 -1.81372937e-01  7.64171369e-01
 -5.23027677e-01 -5.85700844e-01  2.47193163e-01 -2.93478344e-01
  4.98906516e-01 -3.41608906e-01 -2.72283059e-01 -8.71270059e-02
 -1.01984510e-01 -1.68437910e-02 -7.93050252e-01 -6.78542951e-01
 -2.52867106e-01  2.28658831e-01  7.67879690e-02  1.81167688e-01
 -3.95149896e-01 -6.43547988e-01 -5.48744317e-01 -1.07221875e-01
 -2.59122911e-01  7.69842995e-01  4.03534690e-01 -2.81473645e+00
  5.42393541e-01 -1.16019630e-01  1.52413132e+